In [1]:
import csv
import requests
import json
import pandas as pd
import random
import glob
import os
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import pylab
import pingouin as pg
import seaborn as sns


/Users/zeyu/opt/anaconda3/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/Users/zeyu/opt/anaconda3/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.1, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [30]:
#open the full meta data for stories in the ao3 corpus
with open('ao3_analysed_works_all_communities.csv', newline='') as infile:
    #author_meta = list(csv.reader(infile))
    ao3_analysed = pd.read_csv(infile)
ao3_analysed

,fandom,id,Nodes,Edges,Clustering coeff,Reciprocity,Ncomments,Tree Height,Breadth,Breadthest Layer,...,% Single Comments,% Threads long 1 or 2,% Threads longer than 3,% Threads longer than 5,% Threads longer than 10,% Thread where user has last word,Avg % Author Replies in Thread,Avg % Author Replies in Threads longer than 3,Avg % Author Replies in Threads longer than 5,Avg % Author Replies in Threads longer than 10
0,marvel,16769773,17,28,0.0,0.857143,30,3,16,1,...,13.333333,93.750000,6.250000,0.0,0.0,91.666667,48.611111,33.333333,0.000000,0.000000
1,marvel,16704133,12,21,0.0,0.952381,27,4,13,1,...,7.407407,84.615385,15.384615,0.0,0.0,90.909091,48.484848,41.666667,0.000000,0.000000
2,marvel,9821255,24,29,0.0,0.413793,32,2,26,1,...,62.500000,100.000000,0.000000,0.0,0.0,100.000000,50.000000,0.000000,0.000000,0.000000
3,marvel,9417707,17,22,0.0,0.545455,23,3,16,1,...,43.478261,93.750000,6.250000,0.0,0.0,83.333333,47.222222,33.333333,0.000000,0.000000
4,marvel,9340169,22,34,0.0,0.764706,34,2,21,1,...,23.529412,100.000000,0.000000,0.0,0.0,100.000000,50.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173203,warriors,3064727,3,4,0.0,1.000000,492,466,24,1,...,0.000000,0.000000,100.000000,100.0,100.0,95.833333,49.990032,49.990032,49.990032,49.990032
173204,warriors,1775704,16,20,0.0,0.500000,23,3,15,1,...,43.478261,80.000000,20.000000,0.0,0.0,40.000000,40.000000,33.333333,0.000000,0.000000
173205,warriors,1084195,23,22,0.0,0.000000,35,2,34,1,...,94.285714,100.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
173206,warriors,1064463,21,20,0.0,0.000000,21,1,21,1,...,100.000000,100.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [31]:
#merge the dataframe and get the full metadata for the selected stories
with open('mtld_meta_all_ao3.csv', newline='') as infile:
    #author_meta = list(csv.reader(infile))
    mtld_meta_all_ao3 = pd.read_csv(infile)
mtld_meta_all_ao3

,Unnamed: 0,id,fandom,title,authors,date_updated,hits,language,words,mtld
0,58883,4615710,harry_potter,Nemesis,0anon0,20/08/2015,6475.0,English,4950.0,84.342441
1,59183,5154644,harry_potter,The Golden Egg,0anon0,06/11/2015,13888.0,English,2206.0,99.422001
2,57813,2956643,harry_potter,Welcome to the Team,0anon0,30/12/2014,849.0,English,2201.0,101.706190
3,60544,6783052,harry_potter,Leprechaun Gold,0anon0,08/05/2016,2009.0,English,2179.0,82.677102
4,54348,694899,harry_potter,Competition over Christmas,0anon0,27/07/2013,2273.0,English,20081.0,77.402311
...,...,...,...,...,...,...,...,...,...,...
77266,94017,840045,lord_of_the_rings,Dreaming of You,zoop526,26/09/2014,10267.0,English,191428.0,125.640908
77267,94023,788077,lord_of_the_rings,Escaping Isengard,zoop526,20/08/2013,5369.0,English,72963.0,91.700200
77268,94074,689787,lord_of_the_rings,The Healer's Oath,zoop526,03/03/2013,3180.0,English,16911.0,107.593759
77269,93721,2169498,lord_of_the_rings,Zoop Goes to Middle Earth - Part 1,zoop526,29/02/2016,2071.0,English,20230.0,146.330543


In [36]:
h1 = pd.merge(ao3_analysed, mtld_meta_all_ao3, on='id')
h1

,fandom_x,id,Nodes,Edges,Clustering coeff,Reciprocity,Ncomments,Tree Height,Breadth,Breadthest Layer,...,Avg % Author Replies in Threads longer than 10,Unnamed: 0,fandom_y,title,authors,date_updated,hits,language,words,mtld
0,marvel,8075587,13,23,0.000000,0.956522,23,3,11,1,...,0.0,13,marvel,Promises to Keep,Mireille,06/06/2002,2852.0,English,7364.0,89.807188
1,marvel,5097650,14,24,0.000000,0.916667,42,6,20,1,...,0.0,22,marvel,When The Dust Settles,LokiOfSassgaard,02/12/2000,1060.0,English,18805.0,103.837458
2,marvel,4421837,12,23,0.052632,0.956522,36,6,13,1,...,0.0,25,marvel,Those Who Do Not Come Back,spacestationtrustfund,25/10/2014,6948.0,English,2396.0,97.641758
3,marvel,3955135,11,17,0.000000,0.823529,22,5,10,1,...,0.0,32,marvel,Technobiosis,Veldeia,11/01/2009,5086.0,English,16596.0,117.423728
4,marvel,3952471,13,23,0.000000,0.956522,25,3,12,1,...,0.0,35,marvel,Odyssey of the Mind,Veldeia,06/08/2008,3578.0,English,10526.0,100.340002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38039,twilight,423010,10,11,0.000000,0.363636,30,2,28,1,...,0.0,96566,twilight,Figure Eight,tufano79,10/08/2012,4124.0,English,235614.0,91.136256
38040,twilight,421431,6,7,0.000000,0.571429,22,4,13,1,...,0.0,96569,twilight,Love At Twilight,WhisperingKage,11/08/2012,1614.0,English,18324.0,85.079866
38041,twilight,419662,86,90,0.005136,0.000000,127,3,118,1,...,0.0,96575,twilight,The Definition of Want,audreyii_fic,01/06/2012,33360.0,English,56378.0,104.127395
38042,twilight,419651,31,30,0.000000,0.000000,34,1,34,1,...,0.0,96576,twilight,Crossing and Uncrossing Lines,audreyii_fic,01/06/2012,14643.0,English,15882.0,105.377968


In [46]:
#make sure each author still have at least 10 stories
N=10
h1 =h1[h1.groupby('authors')['authors'].transform('count')>N].copy()
h1

,fandom_x,id,Nodes,Edges,Clustering coeff,Reciprocity,Ncomments,Tree Height,Breadth,Breadthest Layer,...,Avg % Author Replies in Threads longer than 10,Unnamed: 0,fandom_y,title,authors,date_updated,hits,language,words,mtld
0,marvel,8075587,13,23,0.000000,0.956522,23,3,11,1,...,0.0,13,marvel,Promises to Keep,Mireille,06/06/2002,2852.0,English,7364.0,89.807188
2,marvel,4421837,12,23,0.052632,0.956522,36,6,13,1,...,0.0,25,marvel,Those Who Do Not Come Back,spacestationtrustfund,25/10/2014,6948.0,English,2396.0,97.641758
3,marvel,3955135,11,17,0.000000,0.823529,22,5,10,1,...,0.0,32,marvel,Technobiosis,Veldeia,11/01/2009,5086.0,English,16596.0,117.423728
4,marvel,3952471,13,23,0.000000,0.956522,25,3,12,1,...,0.0,35,marvel,Odyssey of the Mind,Veldeia,06/08/2008,3578.0,English,10526.0,100.340002
5,marvel,3592419,59,72,0.008998,0.250000,117,4,100,1,...,0.0,41,marvel,Thematic Prompt Shorts,copperbadge,01/04/2013,7287.0,English,4276.0,123.312867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37973,twilight,11650476,33,34,0.006024,0.058824,37,3,35,1,...,0.0,96106,twilight,Technology is Fun,FrankieQuinn13,29/07/2017,7913.0,English,15367.0,117.628022
37993,twilight,3134129,46,47,0.000000,0.085106,65,2,63,1,...,0.0,96223,twilight,Blood and Snow,kirallie,04/08/2019,11785.0,English,18604.0,98.222860
38003,twilight,2309447,15,18,0.000000,0.444444,21,2,16,1,...,0.0,96263,twilight,Her Brothers,Emily_F6,15/09/2014,6931.0,English,16525.0,100.597940
38025,twilight,653560,15,18,0.000000,0.444444,22,3,16,1,...,0.0,96441,twilight,A Hunter's Mate,kirallie,14/09/2015,7038.0,English,33706.0,107.779077


In [47]:
author_lst = h1['authors'].tolist()#.value_counts()
author_lst = set(author_lst)

In [48]:
#________each author's dataframe has been saved as a separate csv file
for name in author_lst:
    name_df = h1.loc[h1['authors'] == f'{name}']
    name_df['date_updated'] = pd.to_datetime(name_df['date_updated'])
    name_df['date_updated'] = name_df['date_updated'].dt.strftime('%Y-%m')
    name_df = name_df.sort_values(by='date_updated')
    name_df['mtld_lead'] = name_df['mtld'].shift(-1)
    name_df = name_df.reset_index()
    name_df["% Author Comments"] = pd.to_numeric(name_df["% Author Comments"], downcast="float")
    name_df['% Author Comments'] = name_df['% Author Comments'].div(100).round(2)
    for i in range(0, len(name_df)):
        name_df.loc[i, 'Num_auth_comt'] = name_df.loc[i, 'Ncomments'] * name_df.loc[i, '% Author Comments']
    name_df['acu_num_auth_comt'] = name_df['Num_auth_comt'].cumsum()
    name_df['acu_num_auth_comt_prev'] = name_df['acu_num_auth_comt'].shift(1)
    name_df['Ncomments_lag'] = name_df['Ncomments'].shift(1)
    name_df['acum_comt']= name_df['Ncomments_lag'].cumsum()
    name_df = name_df.reset_index()
    #name_df.drop(name_df.tail(-1).index, inplace = True)
    name_df.to_csv(str(f'{name}')+'.csv')

   

/var/folders/j_/034d1lj51g3_qcqs0q_kvx900000gn/T/ipykernel_5664/3907933059.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_df['date_updated'] = pd.to_datetime(name_df['date_updated'])
/var/folders/j_/034d1lj51g3_qcqs0q_kvx900000gn/T/ipykernel_5664/3907933059.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name_df['date_updated'] = name_df['date_updated'].dt.strftime('%Y-%m')
